# A Model of an Office

## Imports

In [ ]:
import os
import pickle
from itertools import repeat
from multiprocessing import Pool
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from apsrm import PCRTest, Person, Box, Vaccine
from apsrm.config import POOL_NCORES, DEFAULT_STRAIN
from apsrm.ext.multiprocessing import ProcessSeeder
from apsrm.ext.simulation import (
    create_pathogen,
    run_simulation,
    generate_means_tables,
    plot_histograms)
from utils import (
    BOX_TYPE,
    create_workplace,
    create_emissions_calculator)

pathogen_name = DEFAULT_STRAIN

OUTPUT_BASE_DIR = '../../outputs/office'
OUTPUT_BASE_DIR = OUTPUT_BASE_DIR if os.path.exists(OUTPUT_BASE_DIR) else '.'
OUTPUT_PREFIX = 'stage_1_with_hvac_{}'.format(pathogen_name)
def opath(p): return os.path.join(OUTPUT_BASE_DIR, '{}_{}'.format(OUTPUT_PREFIX, p))

output_pickle = os.path.join(OUTPUT_BASE_DIR, 'all_results.pkl')
run_analyses = not os.path.exists(output_pickle)
process_seeder = ProcessSeeder()

def run_job(workplace, runner, R, intervention_name):
    if run_analyses:
        process_seeder.reset()
        with Pool(POOL_NCORES, initializer=process_seeder) as pool:
            work = pool.imap_unordered(runner, range(R))
            return pd.DataFrame(tqdm(work, total=R)).assign(intervention=intervention_name), intervention_name

def no_reset_standard_runner(*args):
    return run_simulation(workplace, pathogen, emissions_calculator, pcrtest)

def standard_runner(*args):
    workplace.reset(full=True)
    return no_reset_standard_runner(*args)
    
def poc_runner(*args):
    workplace.reset(full=True)
    return run_simulation(workplace, pathogen, emissions_calculator, pcrtest, testing_fraction)

## Values for Scenarios

In [ ]:
R = 192
n_workers = 40
with_hvac = True
force_standard_hvac = True

testing_fraction = 1.
mask_efficiency = .8
external_acph = 1.
hvac_acph = 7.
increased_external_acph = 7.
hvac_return_filtering_efficiency = .85
internal_filtering_efficiency = .95
internal_filtering_volume = 350.

pathogen = create_pathogen(pathogen_name)
emissions_calculator = create_emissions_calculator(pathogen)
pcrtest = PCRTest()

workplace = create_workplace(n_workers=n_workers)

# HVAC System

In [ ]:
if with_hvac:
    hvac_box = Box(1., BOX_TYPE.HVAC, 0, name='hvac') 
    workplace.add_box(hvac_box)

    rooms = np.array([
        [   0.  ,    0.  ,    0.  ,  270.  ,    0.  , 0.],
        [   0.  ,    0.  ,    0.  ,   75.  ,    0.  , 0.],
        [   0.  ,    0.  ,    0.  ,  371.25,    0.  , 0.],
        [ 270.  ,   75.  ,  371.25,    0.  , 1305.  , 0.],
        [   0.  ,    0.  ,    0.  , 1305.  ,    0.  , 0.],
        [   0.  ,    0.  ,    0.  ,    0.  ,    0.  , 0.]])

    exchange_volumes = [hvac_acph * b.volume for b in workplace.boxes]

    hvac_only = np.zeros((len(workplace.boxes), len(workplace.boxes)))
    hvac_only[hvac_box.box_index, :] = hvac_only[:,hvac_box.box_index] = exchange_volumes
    hvac_only[hvac_box.box_index, hvac_box.box_index] = 0.
    hvac_and_rooms = hvac_only + rooms
    
    total_external_ventilation = external_acph * sum(b.volume for b in workplace.boxes if b.use != BOX_TYPE.HVAC)
    default_external_ventilation = np.zeros(len(workplace.boxes))
    default_external_ventilation[hvac_box.box_index] = total_external_ventilation

else:
    default_external_ventilation = [0. if b.use == BOX_TYPE.HVAC else (external_acp * b.volume) for b in workplace.boxes]

def reset_ventilation(**kwargs):
    kwargs['hvac_box_type'] = BOX_TYPE.HVAC
    if 'external_ventilation' not in kwargs:
        kwargs['external_ventilation'] = default_external_ventilation
               
    if with_hvac:
        workplace.set_ventilation_properties(hvac_and_rooms, **kwargs)

    else:
        workplace.set_ventilation_properties(force_standard_hvac_system = True, **kwargs)
               
reset_ventilation()

## Simulations

### BAU

In [ ]:
results_bau = run_job(workplace, standard_runner, R, 'BAU')

### Efect of Masks

Masks are assumed to be 80% effective at preventing the wearer from becoming infected.

In [ ]:
Person.ingestion_filter_efficiency = mask_efficiency
Person.shedding_filter_efficiency = mask_efficiency
results_masks = run_job(workplace, standard_runner, R, 'Masks Only')
Person.ingestion_filter_efficiency = 0.
Person.shedding_filter_efficiency = 0.

### Effect of Random Testing

Note the dramatic change in the time until detection. This has potentially large bearing on the risk in the broader community.

In [ ]:
results_testing = run_job(workplace, poc_runner, R, 'Testing Only')

### Effect of Random Testing and Masks

In [ ]:
Person.ingestion_filter_efficiency = mask_efficiency
Person.shedding_filter_efficiency = mask_efficiency
results_testing_and_masks = run_job(workplace, poc_runner, R, 'Masks and Testing')
Person.ingestion_filter_efficiency = 0.
Person.shedding_filter_efficiency = 0.

### Don't Allow Meetings

In [ ]:
workplace_generators = workplace._generators
workplace._generators = set()
results_no_meetings = run_job(workplace, standard_runner, R, 'No Meetings')
workplace._generators = workplace_generators

### Vaccinate Everyone in the Last Six Months

In [ ]:
vaccine = Vaccine()
def vaccinator(*args):
    workplace.reset(full=True)
    for person, time in zip(workplace.persons, np.random.randint(-180, 0, size=len(workplace.persons))):
        person.vaccinate(time, vaccine)
    return no_reset_standard_runner(*args)
results_vaccinate = run_job(workplace, vaccinator, R, 'Vaccinations')

### Increase Ventilation

In [ ]:
if with_hvac:
    total_external_ventilation = increased_external_acph * sum(b.volume for b in workplace.boxes if b.use != BOX_TYPE.HVAC)
    external_ventilation = np.zeros(len(workplace.boxes))
    external_ventilation[hvac_box.box_index] = total_external_ventilation

else:
    external_ventilation = [0. if b.use == BOX_TYPE.HVAC else (increased_external_acph * b.volume) for b in workplace.boxes]

reset_ventilation(
    external_ventilation = external_ventilation)
results_ventilation = run_job(workplace, standard_runner, R, 'Increased Ventilation')
reset_ventilation()

### HVAC Return Air Filters 

In [ ]:
reset_ventilation(
    hvac_return_filtering_efficiency = hvac_return_filtering_efficiency)
results_hvac_return = run_job(workplace, standard_runner, R, 'HVAC Filters')
reset_ventilation()

### Portable Air Filters

Volume from https://pursuit.unimelb.edu.au/articles/which-air-cleaners-work-best-to-remove-aerosols-that-contain-viruses

In [ ]:
reset_ventilation(
    internal_filtering_volume = internal_filtering_volume,
    internal_filtering_efficiency = internal_filtering_efficiency)
results_portable_filters = run_job(workplace, standard_runner, R, 'Portable Filters')
reset_ventilation()

## Save/Load Results

In [ ]:
if run_analyses:
    all_results = (
        results_bau,
        results_masks,
        results_testing,
        results_testing_and_masks,
        results_no_meetings,
        results_vaccinate,
        results_ventilation,
        results_hvac_return,
        results_portable_filters)
    
    with open(output_pickle, 'wb') as pkl:
        pickle.dump((all_results, R), pkl)
        
else:
    with open(output_pickle, 'rb') as pkl:
        all_results, R = pickle.load(pkl)

infection_counts = pd.concat([r[0] for r in all_results])
dt = CategoricalDtype(categories=[r[1] for r in all_results], ordered=True)
infection_counts['intervention'] = infection_counts['intervention'].astype(dt)

## Plots and Tables

In [ ]:
import matplotlib.font_manager as font_manager

# Add every font at the specified location
font_dir = ['../fnt']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

# Set font family globally
plt.rcParams['font.family'] = 'Palatino'

In [ ]:
plot_histograms(infection_counts, OUTPUT_BASE_DIR, OUTPUT_PREFIX)

In [ ]:
means, means_latex = generate_means_tables(
    infection_counts, R,
    caption = r'Average number of workers infected and average first period in which a case is detected for each intervention considered in stage one when the office has an \ac{hvac} system. The averages are taken over the simulations used to generate the histograms shown in Figures~\ref{fig:stage_1_with_hvac_delta_number_infected} and~\ref{fig:stage_1_with_hvac_delta_period_finished}.',
    label = 'tab:stage1_with_hvac_delta_means')

with open(opath('means.tex'), 'w') as outf: outf.write(means_latex)
print(means)